<a href="https://colab.research.google.com/github/szdziedzic/masked-image-object-recognition/blob/main/fair_detection2_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Make a directory to store the dataset
!mkdir -p datasets/coco

# Download the dataset
!wget http://images.cocodataset.org/zips/train2017.zip
!wget http://images.cocodataset.org/zips/val2017.zip
!wget http://images.cocodataset.org/zips/test2017.zip
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip

# Unzip the downloaded files to the data directory
!unzip -q train2017.zip -d datasets/coco
!unzip -q val2017.zip -d datasets/coco
!unzip -q test2017.zip -d datasets/coco
!unzip -q annotations_trainval2017.zip -d datasets/coco

!rm train2017.zip
!rm val2017.zip
!rm test2017.zip
!rm annotations_trainval2017.zip

--2023-11-26 20:06:25--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.25.108, 54.231.204.241, 52.216.43.1, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.25.108|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘train2017.zip’

train2017.zip       100%[===================>]  18.01G  33.8MB/s    in 9m 20s  

2023-11-26 20:15:45 (32.9 MB/s) - ‘train2017.zip’ saved [19336861798/19336861798]

--2023-11-26 20:15:45--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.214.241, 54.231.171.97, 3.5.9.154, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.214.241|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘val2017.zip’

val2017.zip         100%[===================>] 777

In [1]:
!git clone https://github.com/facebookresearch/detectron2.git

Cloning into 'detectron2'...
remote: Enumerating objects: 15280, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 15280 (delta 0), reused 3 (delta 0), pack-reused 15275
Receiving objects: 100% (15280/15280), 6.17 MiB | 8.18 MiB/s, done.
Resolving deltas: 100% (11111/11111), done.


In [2]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 4.5 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 15.9 MB/s 

In [3]:
!python -m pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 31.0 MB/s eta 0:00:00


In [10]:
!wget https://dl.fbaipublicfiles.com/detectron2/ViTDet/COCO/mask_rcnn_vitdet_b/f325346929/model_final_61ccd1.pkl

--2023-11-26 20:30:27--  https://dl.fbaipublicfiles.com/detectron2/ViTDet/COCO/mask_rcnn_vitdet_b/f325346929/model_final_61ccd1.pkl
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.163.189.96, 3.163.189.108, 3.163.189.51, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.163.189.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 443614688 (423M) [binary/octet-stream]
Saving to: ‘model_final_61ccd1.pkl’

model_final_61ccd1. 100%[===================>] 423.06M  24.7MB/s    in 15s     

2023-11-26 20:30:42 (28.0 MB/s) - ‘model_final_61ccd1.pkl’ saved [443614688/443614688]



In [4]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
torch:  2.1 ; cuda:  cu118
detectron2: 0.6


In [11]:
%run detectron2/tools/lazyconfig_train_net.py --config-file /content/detectron2/projects/ViTDet/configs/COCO/mask_rcnn_vitdet_b_100ep.py --eval-only train.init_checkpoint=model_final_61ccd1.pkl

[11/26 20:32:02 detectron2]: Rank of current process: 0. World size: 1
[11/26 20:32:02 detectron2]: Environment info:
-------------------------------  -----------------------------------------------------------------
sys.platform                     linux
Python                           3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
numpy                            1.23.5
detectron2                       0.6 @/content/detectron2/detectron2
detectron2._C                    not built correctly: No module named 'detectron2._C'
Compiler ($CXX)                  c++ (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
CUDA compiler                    Build cuda_11.8.r11.8/compiler.31833905_0
DETECTRON2_ENV_MODULE            <not set>
PyTorch                          2.1.0+cu118 @/usr/local/lib/python3.10/dist-packages/torch
PyTorch debug build              False
torch._C._GLIBCXX_USE_CXX11_ABI  False
GPU available                    Yes
GPU 0                            Tesla T4 (arch=7.5)
Driver versi

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[11/26 20:32:07 d2.evaluation.evaluator]: Start inference on 5000 batches


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[11/26 20:32:16 d2.evaluation.evaluator]: Inference done 1/5000. Dataloading: 0.3763 s/iter. Inference: 8.2211 s/iter. Eval: 0.0874 s/iter. Total: 8.6867 s/iter. ETA=12:03:44
[11/26 20:32:21 d2.evaluation.evaluator]: Inference done 11/5000. Dataloading: 0.0012 s/iter. Inference: 0.5209 s/iter. Eval: 0.0136 s/iter. Total: 0.5358 s/iter. ETA=0:44:32
[11/26 20:32:26 d2.evaluation.evaluator]: Inference done 21/5000. Dataloading: 0.0016 s/iter. Inference: 0.5218 s/iter. Eval: 0.0111 s/iter. Total: 0.5350 s/iter. ETA=0:44:23
[11/26 20:32:32 d2.evaluation.evaluator]: Inference done 31/5000. Dataloading: 0.0018 s/iter. Inference: 0.5257 s/iter. Eval: 0.0128 s/iter. Total: 0.5406 s/iter. ETA=0:44:46
[11/26 20:32:37 d2.evaluation.evaluator]: Inference done 40/5000. Dataloading: 0.0022 s/iter. Inference: 0.5284 s/iter. Eval: 0.0146 s/iter. Total: 0.5457 s/iter. ETA=0:45:06
[11/26 20:32:43 d2.evaluation.evaluator]: Inference done 50/5000. Dataloading: 0.0022 s/iter. Inference: 0.5308 s/iter. Eval: